In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

In [ ]:
# Retrieve page with the requests module
response = requests.get(url)

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# Examine the results, then determine element that contains sought info
print(soup.prettify())